# Imports

In [ ]:
# import ffmpeg
import cv2
import os
import math
import subprocess as sp
import numpy as np
import matplotlib.pyplot as plt

# Variables

In [ ]:
path="/home/rissu/Desktop/Projects/SlidesExtractor/input"
video_list = os.listdir(path)
print(video_list)
videoFile = path + "/" + video_list[2]
print("Processing", videoFile)



# Comparator Function

In [ ]:
# currently using MSE
# threshold = 36 # gave 136 images
# def ImgCmp(img1,img2):
#     if(img1.shape != img2.shape):
#         print("images shapes inconsistent, please use properly")
#     # both images are numpy array with shape (1080,1920,3)
#     sqsum = (img1-img2)**2
#     sqsum = np.sum(sqsum)
#     sqsum_mean = sqsum/(img1.shape[0] * img1.shape[1]);
#     return sqsum_mean;

# def sigmoid(x):
#     ans =  1/(1+np.exp(-0.0001*x))
#     return ans

# threshold = 50 # gave 136 images
mthreshold = 25
threshold_finder = []
def ImgCmp(img1,img2):
    if(img1.shape != img2.shape):
        print("images shapes inconsistent, please use properly")
    # both images are numpy array with shape (1080,1920,3)
    # I have to make 4 sections of the image
    shape = img1.shape
    split1 = []
    split2 = []
    split1.append(img1[:shape[0]//2, :shape[1]//2])
    split1.append(img1[:shape[0]//2, shape[1]//2:])
    split1.append(img1[shape[0]//2:, :shape[1]//2])
    split1.append(img1[shape[0]//2:, shape[1]//2:])
    split2.append(img2[:shape[0]//2, :shape[1]//2])
    split2.append(img2[:shape[0]//2, shape[1]//2:])
    split2.append(img2[shape[0]//2:, :shape[1]//2])
    split2.append(img2[shape[0]//2:, shape[1]//2:])
    
    outs = []
#     sigmou = []
    for i in range(4):
        sqsum = (split1[i]-split2[i])**2
#         mask = sqsum > 10
#         sqsum = np.multiply(sqsum, mask)
#         sqsum = np.sum(sqsum)
        sqsum_mean = sqsum/(split1[i].shape[0] * split1[i].shape[1]);
#         sigmo = sigmoid(sqsum_mean)
#         sigmo = sqsum_mean/1000
        outs.append(sqsum_mean)
#         sigmou.append(sigmo)
    decider = np.array(outs)
#     marker = np.sum(decider < mthreshold)
#     if marker >= 2:
#         return False
#     decider = np.sum(decider)/4
#     print(np.round(outs,0))
#     print(round(decider,2),marker,end='')
#     threshold_finder.append(decider)
#     if(decider > threshold):
#         return True # this means images are different
#     return False
    return decider
    
def Compare(lastframe,frame,newframe):
    pcmp = ImgCmp(lastframe,frame)
    ncmp = ImgCmp(frame,newframe )
    diff = pcmp-ncmp
#     marker = np.sum(diff < 6)
#     if marker >= 2:
#         return False
    diff = np.sum(diff)/4
    threshold_finder.append(diff)
    print(round(diff,2))
    if(diff > mthreshold):
        return True
    return False

# Extractor

In [ ]:
######## Clean cache
for f in os.listdir("./cache"):
    os.remove("./cache/"+f)
########

count = 0
frame=np.zeros((1080,1920,3))
lastframe=np.zeros((1080,1920,3))
cap = cv2.VideoCapture(videoFile)
frameRate = cap.get(5) #frame rate
total_frames=cap.get(cv2.CAP_PROP_FRAME_COUNT)
print("Frame Rate ", frameRate)
time_interval = 2
img_gap = math.floor(frameRate * time_interval) # ive set image interval to 10sec
if(img_gap == 0): # exception handling
    img_gap = 1
    print("img_gap has become 0, change it appropriately")
    
    
    
######## To figure out the threshold value
# mean_storage = []
########
titty = 0
while(cap.isOpened()):
    titty+=1
    frameId = cap.get(1) #current frame number
    ret, newframe = cap.read()
    if (ret != True):
        break
    if (frameId % img_gap == 0):
        # We got the frame we wanted
        # Do any image processing if you want
        if(Compare(lastframe,frame,newframe)):
            cv2.imwrite("./cache/"+str(count) + ".jpg",lastframe)
            count+=1
            
#         if(Compare(lastframe,frame,newframe)):
#             count+=1
#             cv2.imwrite("./cache/"+str(count) + ".jpg",frame)
#         else:
#             cv2.imwrite("./cache/"+str(count) + ".jpg",frame)
        lastframe = frame
        frame = newframe
#     print(round(titty*100/total_frames,2),"%",end='\r')
cap.release()
print("Total image count ", count)
print ("Done!")

In [ ]:
# def f(a):
#     if abs(a) > 10000:
#         return -100
#     return a
# b = [f(a) for a in threshold_finder]

In [ ]:
# plt.hist(b,100)

In [ ]:
# b = set([int(a) for a in threshold_finder])
# c = {a:0 for a in b}
# for x in threshold_finder:
#     c[int(x)]+=1

In [ ]:

# # Python3 code to demonstrate working of 
# # Sort Dictionary key and values List
# # Using loop + dictionary comprehension
  
# # initializing dictionary
# test_dict = c
# # Sort Dictionary key and values List
# # Using loop + dictionary comprehension
# res = dict()
# for key in sorted(test_dict):
#     res[key] = test_dict[key]
  
# # printing result 
# print(res) 

In [ ]:
# i=0;
# sum = 0;
# k = [a for a in res.keys()]
# while(True):
#     sum += res[k[len(k)-i-1]]
#     print(k[len(k)-i-1], res[k[len(k)-i-1]])
#     i+=1
#     if(sum > 50):
#         break

In [ ]:
#!/usr/bin/env python
# coding: utf-8


### Necessary imports
import cv2
import os
import math
import subprocess as sp
import numpy as np
import sys
from time import sleep
from PIL import Image

### Listing variables
global videoFile
global threshold
global time_interval
videoFiles = []
threshold = 25
time_interval = 2

### Reading arguments

# Remove 1st argument from the
# list of command line arguments
argumentList = sys.argv[1:]

# Options
options = "hm"

# Long options
long_options = ["help", "multiple"]


try:

    if len(argumentList) == 1 and (argumentList[0] not in ["-h","--help"]):
        videoFiles.append(argumentList[0])
    elif argumentList[0] in ["-m","--multiple"]:
        for i in range(1,len(argumentList)):
            videoFiles.append(argumentList[i])
    else:
        print(
            " Usage1: slides_extractor.py {filename}\n",
            "Usage2: slides_extractor.py [OPTION] [VALUE(s)]\n\n",
            "-h, --help : show this help\n",
            "-m, --multiple: multiple files, write names of all the files separated by space"
            )
        sys.exit()
			
except:
	# output error, and return with an error code
    print("Oops", sys.exc_info()[0])
    print ("Invalid command: use ' slides_extractor.py -h ' to get help")
    sys.exit()

def ImgCmp(img1,img2):
    if(img1.shape != img2.shape):
        print("Exception: images' shapes inconsistent")
    # both images are numpy array with shape (1080,1920,3)
    shape = img1.shape
    split1 = []
    split2 = []
    # I have to make 4 sections of the image
    # split1.append(img1[:shape[0]//2, :shape[1]//2])
    # split1.append(img1[:shape[0]//2, shape[1]//2:])
    # split1.append(img1[shape[0]//2:, :shape[1]//2])
    # split1.append(img1[shape[0]//2:, shape[1]//2:])
    # split2.append(img2[:shape[0]//2, :shape[1]//2])
    # split2.append(img2[:shape[0]//2, shape[1]//2:])
    # split2.append(img2[shape[0]//2:, :shape[1]//2])
    # split2.append(img2[shape[0]//2:, shape[1]//2:])

    # just taking the whole image
    split1.append(img1)
    split2.append(img2)

    outs = []
    for i in range(len(split1)):
        sqsum = (split1[i]-split2[i])**2
        sqsum_mean = sqsum/(split1[i].shape[0] * split1[i].shape[1]);
        outs.append(sqsum_mean)
    decider = np.array(outs)
    return decider
    
### Bothway Comparision
def Compare(lastframe,frame,newframe):
    pcmp = ImgCmp(lastframe,frame)
    ncmp = ImgCmp(frame,newframe )
    diff = pcmp-ncmp
    # # attempt to ignore mouse motion
    # marker = np.sum(diff < 3)
    # if marker == 2:
    #     return False
    diff = np.sum(diff)/diff.size
    if(diff > threshold):
        return True
    return False

### Extractor
# Create a cache directory
cache_dir = "cache"
while(not(os.path.isfile("./"+cache_dir))):
    cache_dir+="1"
os.mkdir("./"+cache_dir)

for videoFile in videoFiles:
    print("Starting processing for ",videoFile)
    # cache is expected to be empty
    count = 0
    frame=np.zeros((1080,1920,3))
    lastframe=np.zeros((1080,1920,3))
    cap = cv2.VideoCapture(videoFile)
    frameRate = cap.get(5) #frame rate
    total_frames=cap.get(cv2.CAP_PROP_FRAME_COUNT)
    print("Frame Rate ", frameRate)
    img_gap = math.floor(frameRate * time_interval) # ive set image interval to 10sec
    if(img_gap == 0): # exception handling
        img_gap = 1
        print("Warning: img_gap has become 0, changing it appropriately")
        
        
        
    ## To figure out the threshold value
    # mean_storage = []
    ##
    visited_frames = 0
    while(cap.isOpened()):
        visited_frames+=1
        frameId = cap.get(1) #current frame number
        ret, newframe = cap.read()
        if (ret != True):
            break
        if (frameId % img_gap == 0):
            # We got the frame we wanted
            if(Compare(lastframe,frame,newframe)):
                count+=1
                cv2.imwrite("./"+cache_dir+"/"+str(count) + ".jpg",frame)
            else:
                cv2.imwrite("./"+cache_dir+"/"+str(count) + ".jpg",frame)
            lastframe = frame
            frame = newframe
            print(round(visited_frames*100/total_frames,2),"%",end='\r')
            sys.stdout.flush()
    cap.release()
    print("Total image count ", count)
    print ("Images are extracted in cache, proceeding to create a pdf!")
    image_list = os.listdir("./"+cache_dir)
    print(image_list)
    image_list = [Image.open('./'+cache_dir+"/"+f).convert('RGB') for f in image_list]
    image_list[0].save("./"+videoFile+".pdf", save_all=True, append_images=image_list)
    #clean cache
    for f in os.listdir("./"+cache_dir):
        os.remove("./"+cache_dir+"/"+f)
    os.rmdir("./"+cache_dir)



